# Phase 1
* https://www.kaggle.com/code/divyanshu2000/phase-1-fine-tune-lama-3-2-qlora-lora

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/__results__.html
/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/updated_dataset.csv
/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/__notebook__.ipynb
/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/__output__.json
/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/custom.css
/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/llama-3.2-3b-it-CustomerSupport-ChatBot/adapter_model.safetensors
/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/llama-3.2-3b-it-CustomerSupport-ChatBot/adapter_config.json
/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/llama-3.2-3b-it-CustomerSupport-ChatBot/README.md
/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/llama-3.2-3b-it-CustomerSupport-ChatBot/checkpoint-450/adapter_model.safetensors
/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/llama-3.2-3b-it-CustomerSupport-ChatBot/checkpoint-450/trainer_state.json
/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/llama-3.2-3b-it-CustomerS

# Add input, your Phase 1 code outputs and llama 3.2 model from kaggle

In [2]:
%%capture
%pip install -U bitsandbytes
%pip install transformers==4.44.2
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Merging Tuned Model and Exporting Fine-tuned Llama 3.2 

In [4]:
# Model
base_model_url = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"
new_model_url  = "/kaggle/input/phase1-fine-tune-lama-3-2-qlora-lora/llama-3.2-3b-it-CustomerSupport-ChatBot/" #mention path as i did 

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model_url)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    base_model_url,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Merge adapter with base model

In [6]:
tokenizer.chat_template = None
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [7]:
model = PeftModel.from_pretrained(base_model_reload, new_model_url)

In [8]:
model = model.merge_and_unload()

In [9]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128258, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm

In [11]:
instruction = """You are a top-rated customer service agent named John. 
    Be polite to customers and answer all their questions.
    """

messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": "where do i enter a different shipping address?"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


Thank you for reaching out! I'm here to assist you with updating your shipping address. To make the necessary changes, please follow these steps:

1. Log in to your account on our website.
2. Navigate to the "My Account" or "Profile" section.
3. Look for the "Shipping Addresses" or a similar option.
4. You will find an option to "Add a New Address" or "Edit Shipping Addresses".
5. Choose the option and enter the details of your new shipping address.
6. Save the changes.

If you encounter any difficulties or need further assistance, please don't hesitate to let me know. I'm here to ensure that you have a seamless experience with us.system


Save the tokenizer and model locally. 

In [12]:
new_model = "llama-3.2-3b-it-Ecommerce-ChatBot"

model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('llama-3.2-3b-it-Ecommerce-ChatBot/tokenizer_config.json',
 'llama-3.2-3b-it-Ecommerce-ChatBot/special_tokens_map.json',
 'llama-3.2-3b-it-Ecommerce-ChatBot/tokenizer.json')

Push the tokenizer and merged model to the Hugging Face model repository.

In [ ]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)